# Basic Usage

This notebook demonstrates the standalone usage of `gportal` package.

Make sure to install `gportal` package:

```console
$ pip install gportal
```

Python 3.9 or higher is required.

In [1]:
import gportal

## Search products via Catalogue Service

G-Portal provides [OGC Catalogue Service](https://www.ogc.org/standard/cat/) API.

For API details, see Appendix 7 of [G-Portal User's Manual](https://gportal.jaxa.jp/gpr/assets/mng_upload/COMMON/upload/GPortalUserManual_en.pdf).

### Get dataset IDs

Dataset IDs are required to search products.
You can get them by `gportal.datasets()`.

In [2]:
datasets = gportal.datasets()

The return value is a tree-like dictionary, which structure corresponds to the "spacecraft / sensor" search tree of the G-Portal Web UI, and the leaves are dataset IDs.

In [3]:
datasets.keys()

dict_keys(['GCOM-C/SGLI', 'GCOM-W/AMSR2', 'GPM', 'GPM Constellation satellites', 'GSMaP', 'TRMM_GPMFormat', 'ALOS', 'ALOS-2', 'CIRC', 'ADEOS', 'ADEOS-II', 'AQUA', 'TRMM', 'JERS-1', 'MOS-1', 'MOS-1b', 'NASA-CMR', 'SLATS'])

In [4]:
datasets["GCOM-C/SGLI"]["LEVEL1"]

{'L1A-Visible & Near Infrared, VNR': ['10001000'],
 'L1A-Visible & Near Infrared, POL': ['10001001'],
 'L1A-SWI & TIR': ['10001002'],
 'L1B-Visible & Near Infrared, VNR': ['10001003'],
 'L1B-Visible & Near Infrared, POL': ['10001004'],
 'L1B-SWI & TIR': ['10001005']}

### Build a search query

Let's search GCOM-C L2 SST products near Japan on 2023-03-28.

In [5]:
datasets["GCOM-C/SGLI"]["LEVEL2"]["Oceanic sphere"]["L2-SST"]

['10002002']

In [6]:
res = gportal.search(
    dataset_ids=datasets["GCOM-C/SGLI"]["LEVEL2"]["Oceanic sphere"]["L2-SST"],
    start_time="2023-03-28T00:00:00",
    end_time="2023-03-28T23:59:59",
    bbox=[130, 30, 140, 40],
    params={
        # You can add more parameters.
        # See Appendix 7 of G-Portal User's Manual for details:
        # https://gportal.jaxa.jp/gpr/assets/mng_upload/COMMON/upload/GPortalUserManual_en.pdf
    },
)

`gportal.search` returns `Search` object.

In [7]:
res

<gportal.Search params={'datasetId': '10002002', 'bbox': '130,30,140,40', 'startTime': '2023-03-28T00:00:00', 'endTime': '2023-03-28T23:59:59', 'count': 100}>

`Search.matched` returns the number of matched products.

In [8]:
res.matched()

8

Note that any requests are not invoked with `gportal.search` but only invoked with `Search.matched`.

`Search` class also provides a method to iterate through the resulting products with automatic pagination.

In [9]:
products = list(res.products())
products

[<gportal.Product id=GC1SG1_202303280150A05511_L2SG_SSTDK_3001>,
 <gportal.Product id=GC1SG1_202303280145S05510_L2SG_SSTDK_3001>,
 <gportal.Product id=GC1SG1_202303280145S05510_L2SG_SSTDQ_3001>,
 <gportal.Product id=GC1SG1_202303280150A05511_L2SG_SSTDQ_3001>,
 <gportal.Product id=GC1SG1_202303281303A29303_L2SG_SSTNK_3001>,
 <gportal.Product id=GC1SG1_202303281258S29302_L2SG_SSTNK_3001>,
 <gportal.Product id=GC1SG1_202303281303A29303_L2SG_SSTNQ_3001>,
 <gportal.Product id=GC1SG1_202303281258S29302_L2SG_SSTNQ_3001>]

### Examine metadata of resulting products

`Product` object wraps a GeoJSON in search responses.

In [10]:
product = products[0]
product.geojson

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[125.85595703125, 31.57961463928223],
    [130.72679138183594, 31.06794357299805],
    [133.94281005859375, 30.62568855285645],
    [137.04075622558594, 30.12026596069336],
    [141.29827880859375, 29.29706573486328],
    [142.3202362060547, 29.07710838317871],
    [141.4342498779297, 26.53586006164551],
    [140.59503173828125, 23.98672103881836],
    [139.7974090576172, 21.43053817749023],
    [139.03689575195312, 18.86806488037109],
    [138.3096160888672, 16.29997062683105],
    [137.61216735839844, 13.7268762588501],
    [137.2498321533203, 12.35300827026367],
    [133.02064514160156, 13.08668041229248],
    [130.18714904785156, 13.53771877288818],
    [127.4225082397461, 13.94553470611572],
    [123.5600357055664, 14.46070861816406],
    [122.62117004394531, 14.57615947723389],
    [123.1322250366211, 17.17774772644043],
    [123.635986328125, 19.78003120422363],
    [124.13346099853516, 22.38276863098145],
 

`Product` has shorthand properties to access `properties` dictionary.

In [11]:
print("Granule ID:", product.id)
print("Dataset ID:", product.dataset_id)
print("Observation start:", product.start_time)
print("Observation end", product.end_time)
print("Product URL:", product.data_url)

Granule ID: GC1SG1_202303280150A05511_L2SG_SSTDK_3001
Dataset ID: 10002002
Observation start: 2023-03-28 01:49:44.910000+00:00
Observation end 2023-03-28 01:54:32.480000+00:00
Product URL: https://gportal.jaxa.jp/download/standard/GCOM-C/GCOM-C.SGLI/L2.OCEAN.SST_/3/2023/03/28/GC1SG1_202303280150A05511_L2SG_SSTDK_3001.h5


Let's see the browse image.

In [12]:
from IPython.display import Image

Image(url=product.get_browse_url())

## Download products via SFTP

TODO